In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import numpy as np
import re
import os
from selenium import webdriver

chromedriver = "/Users/Luther/desktop/general assembly/course_materials/classes/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

# driver = webdriver.Chrome(chromedriver)
driver = webdriver.Chrome(executable_path="/Users/Luther/desktop/general assembly/course_materials/classes/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver")

### Creating list of urls to extract the other information

In [2]:
job_urls = []

for i in range(1,11):
    # change below url accordingly to your search. i.e. data analyst to data scientist
    driver.get("https://www.careerbuilder.com/jobs-machine-learning-engineer-in-usa?page_number={}".format(i))
    sleep(3 + 2*np.random.uniform())
    
    html = BeautifulSoup(driver.page_source, 'lxml')
    
    job_links = html.find_all(name='h2', attrs={'class':'job-title show-for-medium-up'})
    
    for link in job_links:
        jd_url = 'https://www.careerbuilder.com{}'.format(link.a['href'])
        job_urls.append(jd_url)

In [3]:
len(job_urls)

250

### Function to extract the fields we need

In [4]:
# creating empty list to store our data
title = []
industry = []
category = []
company = []
location = []
salary = []
details = []
experience = []

In [5]:
def job_details(url):
    
    driver.get(url)
    
    # adjust accordingly to your needs. below code randomly sleeps between 7 to 15secs
    sleep(4 + 3*np.random.uniform())
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # get the job title
    try:
        job_title = soup.find('div', attrs={'class':'small-12 item'}).h1.text.replace('\n','')
        title.append(job_title)
    except AttributeError:
        title.append('NA')
    
    
    # get the industry of the company, e.g. Medical
    try:
        job_industry = soup.find('div', {'class': 'tag', 'id': 'job-industry'}).text.replace('\n','')
        industry.append(job_industry)
    except AttributeError:
        industry.append('NA')
    
    
    # get the nature of the job, e.g. IT
    try:
        job_category = soup.find('div', {'class': 'tag', 'id': 'job-categories'}).text.replace('\n','')
        category.append(job_category)
    except AttributeError:
        category.append('NA')
    
    
    # get name of company
    try:
        company_name = soup.find('div', {'class': 'company-info'}).header.h4.text.replace('\n','')
        company.append(company_name)
    except AttributeError:
        company.append('NA')
    
    
    # get location of the job, e.g. California
    try:
        job_location = soup.find('ul', {'class': 'address'}).li.text.replace('\n','')
        location.append(job_location)
    except AttributeError:
        location.append('NA')
    
    

    # get salary data
    job_salary = soup.find_all('div', {'class': 'tag'})

    count = 0
    for i in job_salary:
        if '$' in i.text:
            count += 1
            salary.append(i.text.replace('\n',''))
            break

    if count == 0:
        salary.append('NA')
    
    
    # get job description and requirements
    job_description = soup.find_all(name='div', attrs={'class':'description'})
    
    description = ''
    for i in job_description:
        description += i.text
    
    details.append(description)
    
    
    # attempt to get experience required from main description. 
    # will not extract if its written in format not covered by below 
    try:
        exp = re.search(r'[1-9]?(-| - | to )*[1-9]\+? [Yy]ears?',description).group(0)
        experience.append(exp)
    except AttributeError:
        experience.append('NA')
    
    return

In [6]:
# extracting the required fields. to print the iteration number so we know where is it now
for i in job_urls:
    print job_urls.index(i)
    job_details(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [7]:
# check to ensure count is correct
print len(title)
print len(industry)
print len(category)
print len(company)
print len(location)
print len(details)
print len(salary)
print len(experience)


250
250
250
250
250
250
250
250


In [8]:
data_dict = {'title': title, 'industry': industry, 'category': category, 'company': company, 'location': location, 'details': details, 'salary': salary, 'experience': experience}

In [9]:
import pandas as pd

In [10]:
jobs = pd.DataFrame(data_dict)

In [11]:
jobs.head(10)

,category,company,details,experience,industry,location,salary,title
0,"Engineering, Information Technology, Design",Robert Half Technology,\nRef ID: 04380-0010374687Classification: Soft...,65 years,"Computer Hardware, Computer Software","Dallas, TX 75201",$156.75 - $181.50 /Hour,Machine Learning Engineer
1,"Information Technology, Engineering, Professio...",CyberCoders,\nThis position is open as of 4/22/2018.Leadin...,NA,Industrial,"Los Angeles, CA 90001","$120,000.00 - $160,000.00 /Year",Data Engineer/Scientist
2,"Construction, Engineering",Aerotek,\nRobotics Perception Software EngineerJob Dut...,5 years,"Computer Hardware, Computer Software, Construc...","PITTSBURGH, PA",$30.00 - $55.00 /Hour,Project Engineer
3,"Engineering, Information Technology, Design",CyberCoders,\nThis position is open as of 4/22/2018.Deep L...,NA,Computer Software,"Santa Clara, CA 95050","$130,000.00 - $160,000.00 /Year","Deep Learning, Computer Vision, Algorithm Engi..."
4,Information Technology,"Intelliswift Software, Inc.",\nOperations Research/Machine Learning Data Sc...,NA,Internet - ECommerce,"Sunnyvale, CA",NA,Machine Learning Engineer / Data Scientist
5,Information Technology,"Pinnacle Partners, Inc",\n\nMachine Learning Engineer\nUp to $120k per...,NA,Other Great Industries,"Indianapolis, IN","$0.00 - $120,000.00 /Year",Machine Learning Engineer
6,Information Technology,"American Cybersystems, Inc. (ACS Group)",\n\n\n\n\n\n Job Title: (IDS) Data Scientist \...,NA,Other Great Industries,"Newport News, VA",$50.00 - $70.00 /Hour,Data Scientist 2
7,Information Technology,Genesis10,\nGenesis10 is currently seeking a Software En...,5 years,Entertainment,"Mountain View, CA",$69.51 - $76.03 /Hour,Software Engineer
8,"Information Technology, Engineering, Professio...",CyberCoders,\nThis position is open as of 4/22/2018.Senior...,5+ years,Computer Software,"Santa Monica, CA 90401","$120,000.00 - $140,000.00 /Year",Lead Computer Vision Engineer
9,"Information Technology, Engineering, Professio...",CyberCoders,\nThis position is open as of 4/22/2018.Python...,6+ years,Computer Software,"San Francisco, CA 94102","$150,000.00 - $200,000.00 /Year",Sr. Python Data Engineer


In [12]:
jobs.to_csv('machine_learning_engineer.csv', encoding='utf-8')

In [13]:
driver.close()